In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
#sns.set_context("poster")

<img src=metrics_1.png>

<img src=metrics_2.png>

imágenes son esenciales, know by heart dice profesor (he hecho foto por si acaso)
false positives ej. cáncer.
recall de los que has categorizado como positivo, dividido entre todos los positivos que existen.
precision: seleccionamoscuáles son correctos de todos los que hemos seleccionado. 

precision and recall are the basic ones.
si es la primera vez que pruebas, priorizas recall... you don't want to miss the people who have cancer. e.j.98% recall even if precision is really bad.
haces segundo test de los seleccionados? entonces quieres precision. 
para netflix le da igual wrong recommendaiton, por lo que le importa recall, catch as many as you can - all the styles and tastes of the people.
precision para pieza de coche, porque sino tendrás devoluciones por instalar piezas que no estaban bien.

por lo que muchas veces no quieres equally precision and recall.


then normally also talk about accuracy (is the middle measure, we normally work with it). (como hasta ahora). good medium measure. but depending on what you are doing, it is not as important.
de los que acertamos entre todo. 

changing the threshold will change the balance of precision and recall.

recall is also called sensitivity. is the same . but you see this terminology in doctors. biomedical sectors. suelen ir primero por recall y luego specificity, y quizá por en medio algo de precision.

specificity es el opuesto de recall (1- recall). si has dicho que esa persona no tiene cáncer, asegurarte que no tiene. 

<h1> Performance Metrics </h1>

The choice of metrics for evaluating a model is of the utmost importance. We will tune the model, select hyperparameters and choose a particular algorithm among others, according to this metrics. 

In this notebook we will focus on two of the main machine learning problems and we will discuss the metrics associated with them:
<blockquote>
        <ul style="list-style-type:none;">
            <li> <b>1) Classification problems.</b> We will continue to use the Pima Indias onset diabetes dataset that we have been using so far. In this dataset all attributes are numeric and its a binary classification problem.</li> 
            <br>
            <li> <b>2) Regression problems.</b> For regression we will use also a very traditional dataset, the Boston House Price. Again all input variables are numeric.</li>
        </ul>
</blockquote>

Obviously not all problems in machine learning can be categorized as regression or classification problems, we also have clustering, association rules, topic modeling, etc... However, classification and regression remain as the most substantial ones. 

# CLASSIFICATION METRICS

Classification is probably the most common problem in machine learning and many problems can be reduced to a classification problem. 

Here, we will review the following metrics:

            1) Accuracy. 
            2) Logarithmic loss.
            3) Area under ROC curve.
            4) Confusion matrix.
            5) Classification Report. 
        
using our already familiar Pima Indians diabetes dataset. 


<img src="Pima_indians_cowboy_1889.jpg">

In this exercise we will use one of the traditional Machine Learning dataset, the Pima Indians diabetes dataset.

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

Content
The datasets consists of several medical predictor variables and one target variable, <b>Outcome</b>. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.
<blockquote>
        <ul style="list-style-type:square;">
            <li>Pregnancies</li> 
            <li>Glucose</li>
            <li>BloodPressure</li>
            <li>SkinThickness</li>
            <li>Insulin</li>
            <li>BMI</li>
            <li>DiabetesPedigreeFunction (scores de likelihood of diabetes based on family history)</li>
            <li>Age</li>
            <li>Outcome</li>
        </ul>
</blockquote>

La imagen útil de precision, etc la encontramos en wikipedia. útil tenerla a mano.
ROC es muy popular, se ve en todas partes. be very careful. AUC. lo utilizan como accuracy?
No puedes comparar two classifiers the number is not representative. Only compare if it's same classifier.
ROC is way over used.

In [2]:
# Load the Pima indians dataset and separate input and output components 

from numpy import set_printoptions
set_printoptions(precision=3)

filename="pima-indians-diabetes.data.csv"
names=["pregnancies", "glucose", "pressure", "skin", "insulin", "bmi", "pedi", "age", "outcome"]
p_indians=pd.read_csv(filename, names=names)
p_indians.head()

# First we separate into input and output components
array=p_indians.values
X=array[:,0:8]
Y=array[:,8]
np.set_printoptions(suppress=True)
X
pd.DataFrame(X).head()

,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

,0,1,2,3,4,5,6,7
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0


<h1>Classification Accuracy </h1>

Accuracy is the ratio of correct predictions over all predictions. It is by far the most used metrics. 

However, it is only suitable when classes are balanced and errors in each class are equally important. For example whe it is equally important to missclassify a healthy person as having cancer (Type I) or a sick one as healty (Type II). As you can guess, in many cases this is not the case and accuracy is many times misused. 


In [3]:
# Accuracy

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

p_indians.head()

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)
scoring="accuracy"


#Logistic regression
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

print(f'Logistic regression, k-fold {splits:d} - Accuracy {results.mean()*100:.3f}% ({results.std()*100:.3f}%)')

# A: en vex de accuracy puedo poner precision como otra medida a mirar.

,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Logistic regression, k-fold 10 - Accuracy 76.951% (4.841%)


<h1>Logarithmic Loss</h1>

Logarithmic loss (or logloss) is a performance metric for evaluating predictions of probabilities of membership to a clas as a scalar between 0 and 1 that is seen as a measure of confidence. 

Correct and incorrect predictions are rewarded or punished according to the confidence on the prediction. 
Each predicted probability is compared to the actual class output value (0 or 1) and a score is calculated that penalizes the probability based on the distance from the expected value.The penalty is logarithmic, offering a small penalty for small differences (0.1 or 0.2) and a large one for a large difference (0.9 or 1.0).

cross_val_score inverts (expressing the inversion with a - sign) the measure, therefore 0 is better. 

In [4]:
# Logarithmic Loss

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

p_indians.head()

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)
scoring="neg_log_loss"

#Logistic regression
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

print(f'Logistic regression, k-fold {splits:d} - Logloss {results.mean():5.3f} ({results.std():5.3f})')


,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Logistic regression, k-fold 10 - Logloss -0.493 (0.047)


<img src="AUC.png">

<h1>Area under ROC curve</h1>

Area under the ROC curve (AUC for short) is a metric used in binary classifications. 

It represents the ability of the model to discriminate between positive and negative classes. An AUC of 0.5 represents a model that is as good as random, while an area of 1 represents a perfect model. 

The area under the curve can be broken down into two metrics: sentitivity (recall) and specificity. In general any binary classification problem is a tradeoff between these two measures. 

        - Sensitivity (True positive rate or recall). Represents the number of instances of the positive class that were correctly classified. 
        - Specificity (True negative rate). Number of instances of the negative class classified correctly. 


In [5]:
# Area under ROC curve

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

p_indians.head()

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)
scoring="roc_auc"

#Logistic regression
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

print(f'Logistic regression, k-fold {splits:d} - AUC {results.mean():5.3f} ({results.std():5.3f})')


,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Logistic regression, k-fold 10 - AUC 0.824 (0.041)


<img src="confusionMatrix.jpg">

<h1>Confusion Matrix </h1>

The confusion matrix is a presentation of the accuracy of the model in its four classes: True Positives, False Positives, False Negatives and True Negatives. 

From it we can easily derive Precision, Recall, Specificity and Accuracy. 


In [6]:
# Confusion Matrix

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix

p_indians.head()

test_size=0.3
seed=7

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

c_matrix=confusion_matrix(Y_test, Y_predicted)

print("Confusion Matrix")
print(c_matrix)

print()
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}') # esto automáticamente ya te hace accuracy
print(f'Accuracy check with conf. matrix {(c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()*100:.5f}') # aquí lo haces a mano.




,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Confusion Matrix
[[130  17]
 [ 38  46]]

Accuracy 76.19048
Accuracy check with conf. matrix 76.19048


<h1>Classification Report</h1>

The scikit-learn library provides a report that is convenient and useful in terms of summarizing many of the measures that are commonly used together with accuracy. 

For each class it presents:
<blockquote>
1) Precision. $Precision = \frac{True Positives}{True Positives + False Posities}$

2) Recall (also known as Sensitivity). $Recall = \frac{True Positives}{True Positivies + False Negatives}$

3) F1 score. It's the harmonic mean of precision and recall. $F1 = 2 ·\frac{precision · recall}{precision + recall}$    , provides a good balance between precision and recall when classes are unevenly distributed. Best is 1, worst is 0.   

4) Support. It's the number of elements of each class in Y_test.

</blockquote>
The reported averages include:

        1) Macro average. Averaging the unweighted mean per label. 
        2) Weighted average. Averaging the support-weighted mean per label. 
        3) Sample average. Only for multilabel classification. 
        4) Micro average. Accuracy for a binary classification. Averaging the total true positives, false negatives and false positives.

In [7]:
# Classification Report 

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

p_indians.head()

test_size=0.3
seed=7

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

report = classification_report(Y_test, Y_predicted, digits=5) # digits cuántos decimales quieres.


print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}')
print()
print(report)



,pregnancies,glucose,pressure,skin,insulin,bmi,pedi,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 76.19048

              precision    recall  f1-score   support

         0.0    0.77381   0.88435   0.82540       147
         1.0    0.73016   0.54762   0.62585        84

    accuracy                        0.76190       231
   macro avg    0.75198   0.71599   0.72562       231
weighted avg    0.75794   0.76190   0.75283       231



<b><font color="red" size=6>Mission 1</font>

We will use our predictions for top-10 and top-50 the Shanghai and Times Dataset
<br><br>
a) For the Shanghai dataset evaluate accuracy, logloss, AUC, confusion matrix and classification report. Briefly discuss the diferences. 
<br><br>
b) Same for the Times ranking. 
<br><br>

</b>

In [8]:
# A) For the Shanghai dataset evaluate accuracy, logloss, AUC, confusion matrix and classification report. 
# ... Briefly discuss the diferences. 

# Import dataset (code copy-pasted from my previous notebook)
sg_data = pd.read_csv('shanghaiData.csv')
sg_data['world_rank'] = sg_data['world_rank'].str.split('-').str.get(0).astype(float)
sg_data.drop(columns=['national_rank','total_score'], inplace=True)
sg_data.dropna(inplace=True)
sg_data.head()


print('------------------------------------------- ACCURACY ------------------------------------------------------------')
print('------------------------------------------- ACCURACY TOP10 ------------------------------------------------------')
# Imports (not necessary, because it is already above)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)

# Select input and target variables and transform to numpy arrays.
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<11
Y = sg_data['Top10'].values

#Logistic regression
scoring="accuracy"
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

#Show results
print(f'Logistic regression TOP10, k-fold {splits:d} - Accuracy {results.mean()*100:5.3f}% ({results.std()*100:5.3f}%)')
print('------------------------------------------- ACCURACY TOP50 ------------------------------------------------------')
# Select input and target variables and transform to numpy arrays.
X = sg_data.iloc[:,2:8].values
sg_data['Top50'] = sg_data['world_rank']<51
Y = sg_data['Top50'].values

#Logistic regression
scoring="accuracy"
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

#Show results
print(f'Logistic regression TOP50, k-fold {splits:d} - Accuracy {results.mean()*100:5.3f}% ({results.std()*100:5.3f}%)')

print() # esto es un enter en jupyter notebook
print('------------------------------------------- LOGLOSS ------------------------------------------------------------')
print('------------------------------------------- LOGLOSS TOP10 ------------------------------------------------------')
# Imports and KFold (not necessary, because it is already above)
# KFold
splits=5 # I am using less than 10 folds, otherwise impacted because I am doing top10, then the first is all 1, the rest 0.
kfold=KFold(n_splits=splits, random_state=7)
# Select input and target variables and transform to numpy arrays.
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<11 # this is boolean, I need it in 1 and 0 for log regression!!!!!!!!
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values # transform to numpy array

#Logistic regression
scoring="neg_log_loss" 
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
#from sklearn.metrics import log_loss
#model.fit(X, Y)
#a = model.predict(X)
#log_loss(Y, a)
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

# Show
print(f'Logistic regression, k-fold {splits:d} - Logloss {results.mean():5.3f} ({results.std():5.3f})')

print('------------------------------------------- LOGLOSS TOP50 ------------------------------------------------------')
# Imports and KFold (not necessary, because it is already above)
# KFold
splits=5 # I am using less than 10 folds, otherwise impacted because I am doing top10, then the first is all 1, the rest 0.
kfold=KFold(n_splits=splits, random_state=7)
# Select input and target variables and transform to numpy arrays.
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<51 #change this
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values 

#Logistic regression
scoring="neg_log_loss" 
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

# Show
print(f'Logistic regression, k-fold {splits:d} - Logloss {results.mean():5.3f} ({results.std():5.3f})')

print()
print('------------------------------------------- AUC ------------------------------------------------------------')
print('------------------------------------------- AUC TOP10 ------------------------------------------------------')
# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# Select variables
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<11 
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values
# KFold
splits=5
kfold=KFold(n_splits=splits, random_state=7)
scoring="roc_auc"
#Logistic regression
model = LogisticRegression(solver='liblinear')
# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=KFold(n_splits=splits, random_state=7))

print(f'Logistic regression, k-fold {splits:d} - AUC {results.mean():5.3f} ({results.std():5.3f})')

print('------------------------------------------- AUC TOP50 ------------------------------------------------------')
# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# Select variables
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<51 # changed this, copy-pasted from top10
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values
# KFold
splits=5
kfold=KFold(n_splits=splits, random_state=7)
scoring="roc_auc"
#Logistic regression
model = LogisticRegression(solver='liblinear')
# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=KFold(n_splits=splits, random_state=7))

print(f'Logistic regression, k-fold {splits:d} - AUC {results.mean():5.3f} ({results.std():5.3f})')

print()
print('------------------------------------------- CONFUSION MATRIX --------------------------------------------------------')
print('------------------------------------------- CONFUSION MATRIX TOP10 --------------------------------------------------')
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Select variables (not necessary since I have them defined above, but to practice)
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<11
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values

# Select arguments, split the data, log regression, confusion matrix
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
Y_predicted = model.predict(X_test)
c_matrix=confusion_matrix(Y_test, Y_predicted)

# Show
print("Confusion Matrix")
print(c_matrix)

print()
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}') # esto automáticamente ya te hace accuracy
print(f'Accuracy check with conf. matrix {(c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()*100:.5f}') # aquí lo haces a mano.

print('------------------------------------------- CONFUSION MATRIX TOP50 --------------------------------------------------')
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Select variables (not necessary since I have them defined above, but to practice)
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<51 # changed this, copy-pasted from top10
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values

# Select arguments, split the data, log regression, confusion matrix
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
Y_predicted = model.predict(X_test)
c_matrix=confusion_matrix(Y_test, Y_predicted)

# Show
print("Confusion Matrix")
print(c_matrix)

print()
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}') # esto automáticamente ya te hace accuracy
print(f'Accuracy check with conf. matrix {(c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()*100:.5f}') # aquí lo haces a mano.


print()
print('------------------------------------------- CLASSIFICATION REPORT ----------------------------------------------------')
print('------------------------------------------- CLASSIFICATION REPORT TOP10 ----------------------------------------------')
# Classification Report 

# Imports (although already above, just for practice)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Select variables
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<11
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values

# Set the parameters for the train_test_split
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Logistic Regression
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

# Classification report
report = classification_report(Y_test, Y_predicted, digits=5) # digits cuántos decimales quieres.

# Accuracy metric
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}')
print()
print(report)

print('------------------------------------------- CLASSIFICATION REPORT TOP50 ----------------------------------------------')
# Classification Report 

# Imports (although already above, just for practice)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Select variables
X = sg_data.iloc[:,2:8].values
sg_data['Top10'] = sg_data['world_rank']<51 # changed this, copy-pasted the rest from classification report top10
dicc={True:1,False:0}
Y = sg_data['Top10'].replace(dicc)
Y = Y.values

# Set the parameters for the train_test_split
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Logistic Regression
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

# Classification report
report = classification_report(Y_test, Y_predicted, digits=5)

# Accuracy metric
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}')
print()
print(report)

print()
# DISCUSS DIFFERENCES (refer to next cell!!!!)
print("Discussion of differences in the cell below")

,world_rank,university_name,alumni,award,hici,ns,pub,pcp,year
0,1.0,Harvard University,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2.0,University of Cambridge,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3.0,Stanford University,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4.0,"University of California, Berkeley",71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5.0,Massachusetts Institute of Technology (MIT),74.0,80.6,66.7,65.8,64.3,53.0,2005


------------------------------------------- ACCURACY ------------------------------------------------------------
------------------------------------------- ACCURACY TOP10 ------------------------------------------------------
Logistic regression TOP10, k-fold 10 - Accuracy 99.836% (0.179%)
------------------------------------------- ACCURACY TOP50 ------------------------------------------------------
Logistic regression TOP50, k-fold 10 - Accuracy 98.687% (0.307%)

------------------------------------------- LOGLOSS ------------------------------------------------------------
------------------------------------------- LOGLOSS TOP10 ------------------------------------------------------
Logistic regression, k-fold 5 - Logloss -0.009 (0.005)
------------------------------------------- LOGLOSS TOP50 ------------------------------------------------------
Logistic regression, k-fold 5 - Logloss -0.042 (0.009)

------------------------------------------- AUC -----------------------------

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Confusion Matrix
[[1435    0]
 [   4   24]]

Accuracy 99.72659
Accuracy check with conf. matrix 99.72659
------------------------------------------- CONFUSION MATRIX TOP50 --------------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Confusion Matrix
[[1294    9]
 [   9  151]]

Accuracy 98.76965
Accuracy check with conf. matrix 98.76965

------------------------------------------- CLASSIFICATION REPORT ----------------------------------------------------
------------------------------------------- CLASSIFICATION REPORT TOP10 ----------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 99.72659

              precision    recall  f1-score   support

           0    0.99722   1.00000   0.99861      1435
           1    1.00000   0.85714   0.92308        28

    accuracy                        0.99727      1463
   macro avg    0.99861   0.92857   0.96084      1463
weighted avg    0.99727   0.99727   0.99716      1463

------------------------------------------- CLASSIFICATION REPORT TOP50 ----------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 98.76965

              precision    recall  f1-score   support

           0    0.99309   0.99309   0.99309      1303
           1    0.94375   0.94375   0.94375       160

    accuracy                        0.98770      1463
   macro avg    0.96842   0.96842   0.96842      1463
weighted avg    0.98770   0.98770   0.98770      1463


Discussion of differences in the cell below


------------------------------------------------------------------------------------------------------------------------------
-------------------------------------DISCUSS DIFFERENCES---------------------------------------------------------------------

As expected, the ACCURACY of the logistic regression when predicting Top10 is higher than for Top50.
The LOGLOSS obtained (more negative for Top50 than for Top10) indicates that the mistakes in the Top10 have been bigger/worse, than the mistaked when predicting the ones in the Top50. Since it is a metric that tells you that from the ones that from the failed predictions, for how much/how bad it has been.
The AUC for the Top10 shows an incredible area of 1, which may seem impossible, but it is not if we check the CONFUSION MATRIX. AUC measures the ratio of FP (sensitivity), and since in this prediction (with this seed, etc.) I have 0 FP, the area is 1.
The area for the Top50 is less than one and far above the 0.5 of the random curve, so it is fine.
The CONFUSION MATRIX allows to check the number of TP, FP, FN and TN and the accuracy, which of course decreases from the Top10 prediction to the Top50 (because of the size of the dataset and the amount of predictions). Finally, the CLASSIFICATION REPORT provides more metrics that complement the analysis of the Confusion Matrix.

In [9]:
# B) Same for the Times ranking. 
# ... We will use our predictions for top-10 and top-50 the Times Dataset 
#... and evaluate accuracy, logloss, AUC, confusion matrix and classification report. Briefly discuss the diferences. 


# Import dataset and do corresponding adjustments. (copy-pasted from the 'Feature Selection' notebook)
times = pd.read_csv('timesData.csv')
times.world_rank.replace('=.','.', regex=True, inplace=True)
times['world_rank'] = times['world_rank'].str.split('-').str.get(0).astype(float)
times.world_rank = pd.to_numeric(times.world_rank, errors='coerce')
times.dropna(subset=['world_rank'], axis=0, inplace=True) # DROPNA only of 'world_rank'
times.international = pd.to_numeric(times.international, errors='coerce')
times.income = pd.to_numeric(times.income, errors='coerce')
times.num_students.replace('\D','', regex=True, inplace=True)
times.num_students = pd.to_numeric(times.num_students, errors='coerce')
times.international_students.replace('\D','', regex=True, inplace=True)
times.international_students = pd.to_numeric(times.international_students, errors='coerce')
times.international_students = times.international_students/100
times['females'] = times.female_male_ratio.str.split(':').str.get(0) # manipulate so I can use the data of this column.
times['males'] = times.female_male_ratio.str.split(':').str.get(1)
times.females = pd.to_numeric(times.females, errors='coerce')
times.males = pd.to_numeric(times.males, errors='coerce')
times['ratio_male_to_female'] = times.males/times.females
times.drop(columns='total_score',inplace=True)
times.dropna(inplace=True)
times.head()

print('------------------------------------------- ACCURACY ------------------------------------------------------------')
print('------------------------------------------- ACCURACY TOP10 ------------------------------------------------------')

# Imports (although not necessary because they have already been imported previously in this notebook)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Select input and target variables and transform to numpy arrays.
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<11
Y = times['Top10'].values

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)

#Logistic regression
scoring="accuracy"
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

#Show results
print(f'Logistic regression TOP10, k-fold {splits:d} - Accuracy {results.mean()*100:5.3f}% ({results.std()*100:5.3f}%)')

print('------------------------------------------- ACCURACY TOP50 ------------------------------------------------------')

# Imports (although not necessary because they have already been imported previously in this notebook)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Select input and target variables and transform to numpy arrays.(same as before)
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<51 # changed from 'Accuracy top10', the rest copy-pasted
Y = times['Top10'].values

# KFold
splits=10
kfold=KFold(n_splits=splits, random_state=7)

#Logistic regression
scoring="accuracy"
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

#Show results
print(f'Logistic regression TOP10, k-fold {splits:d} - Accuracy {results.mean()*100:5.3f}% ({results.std()*100:5.3f}%)')

print()

print('------------------------------------------- LOGLOSS ------------------------------------------------------------')
print('------------------------------------------- LOGLOSS TOP10 ------------------------------------------------------')
# Imports and KFold (not necessary, because it is already above)
# KFold
splits=4 # I am using less than 10 folds, otherwise impacted because I am doing top10, then the first is all 1, the rest 0.
#el número de splits tiene que ser divisible por el número de casos, hecho len(Y) y sí es posible entre 2
# divido en 4 el data set, tengo el 25% en cada uno.
kfold=KFold(n_splits=splits, random_state=6)
# Select input and target variables and transform to numpy arrays.
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<11 # changed from 'Accuracy top10', the rest copy-pasted
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

#Logistic regression
scoring="neg_log_loss" 
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy ;; I ws trying with try/except until I got the result. 
# I leave it here as a reminder for me of how it is done.
try:
    results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

    # Show
    print(f'Logistic regression, k-fold {splits:d} - Logloss {results.mean():5.3f} ({results.std():5.3f})')
except:
    pass

print('------------------------------------------- LOGLOSS TOP50 ------------------------------------------------------')
# Imports and KFold (not necessary, because it is already above)
# KFold
splits=4 # I am using less than 10 folds, otherwise impacted because I am doing top10, then the first is all 1, the rest 0.
#el número de splits tiene que ser divisible por el número de casos, hecho len(Y) y sí es posible entre 2
# divido en 4 el data set, tengo el 25% en cada uno.
kfold=KFold(n_splits=splits, random_state=6)
# Select input and target variables and transform to numpy arrays.
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<51 # changed from 'Accuracy top10', the rest copy-pasted
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

#Logistic regression
scoring="neg_log_loss" 
model = LogisticRegression(solver='liblinear')

# Obtain the performance measure - accuracy 
results = cross_val_score(model, X, Y, scoring=scoring, cv=kfold)

# Show
print(f'Logistic regression, k-fold {splits:d} - Logloss {results.mean():5.3f} ({results.std():5.3f})')

print()

print('------------------------------------------- AUC ------------------------------------------------------------')
print('------------------------------------------- AUC TOP10 ------------------------------------------------------')
# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# Select variables
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<11
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values
# KFold
splits=4
kfold=KFold(n_splits=splits, random_state=7)
scoring="roc_auc"
#Logistic regression
model = LogisticRegression(solver='liblinear')
# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=KFold(n_splits=splits, random_state=7))

print(f'Logistic regression, k-fold {splits:d} - AUC {results.mean():5.3f} ({results.std():5.3f})')

print('------------------------------------------- AUC TOP50 ------------------------------------------------------')
# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
# Select variables
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<51 #copy-pasted code from 'AUC TOP10' and just changed this value
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values
# KFold
splits=4
kfold=KFold(n_splits=splits, random_state=7)
scoring="roc_auc"
#Logistic regression
model = LogisticRegression(solver='liblinear')
# Obtain the performance measure - accuracy
results = cross_val_score(model, X, Y, scoring=scoring, cv=KFold(n_splits=splits, random_state=7))

print(f'Logistic regression, k-fold {splits:d} - AUC {results.mean():5.3f} ({results.std():5.3f})')

print()
print('------------------------------------------- CONFUSION MATRIX --------------------------------------------------------')
print('------------------------------------------- CONFUSION MATRIX TOP10 --------------------------------------------------')
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Select variables (not necessary since I have them defined above, but to practice)
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<11 
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

# Select arguments, split the data, log regression, confusion matrix
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
Y_predicted = model.predict(X_test)
c_matrix=confusion_matrix(Y_test, Y_predicted)

# Show
print("Confusion Matrix")
print(c_matrix)

print()
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}') # esto automáticamente ya te hace accuracy
print(f'Accuracy check with conf. matrix {(c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()*100:.5f}') # aquí lo haces a mano.

print('------------------------------------------- CONFUSION MATRIX TOP50 --------------------------------------------------')
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Select variables (not necessary since I have them defined above, but to practice)
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<51 #copy-pasted code from 'Confusion Matrix TOP10' and just changed this value
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

# Select arguments, split the data, log regression, confusion matrix
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
Y_predicted = model.predict(X_test)
c_matrix=confusion_matrix(Y_test, Y_predicted)

# Show
print("Confusion Matrix")
print(c_matrix)

print()
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}') # esto automáticamente ya te hace accuracy
print(f'Accuracy check with conf. matrix {(c_matrix[0,0]+c_matrix[1,1])/c_matrix.sum()*100:.5f}') # aquí lo haces a mano.

print()
print('------------------------------------------- CLASSIFICATION REPORT ----------------------------------------------------')
print('------------------------------------------- CLASSIFICATION REPORT TOP10 ----------------------------------------------')
# Classification Report 

# Imports (although already above, just for practice)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Select variables
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<11 
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

# Set the parameters for the train_test_split
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Logistic Regression
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

# Classification report
report = classification_report(Y_test, Y_predicted, digits=5) # digits cuántos decimales quieres.

# Accuracy metric
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}')
print()
print(report)

print('------------------------------------------- CLASSIFICATION REPORT TOP50 ----------------------------------------------')
# Classification Report 

# Imports (although already above, just for practice)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Select variables
X = pd.concat([times.iloc[:,3:11],times['ratio_male_to_female']],axis=1).values
times['Top10'] = times['world_rank']<51 # copy-pasted from 'Classification Report TOP10', just changed this value
dicc={True:1,False:0}
Y = times['Top10'].replace(dicc)
Y = Y.values

# Set the parameters for the train_test_split
test_size=0.3
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Logistic Regression
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)

Y_predicted = model.predict(X_test)

# Classification report
report = classification_report(Y_test, Y_predicted, digits=5)

# Accuracy metric
print(f'Accuracy {model.score(X_test, Y_test)*100:.5f}')
print()
print(report)

print()
# DISCUSS DIFFERENCES (refer to next cell!!!!)
print("Discussion of differences in the cell below")

,world_rank,university_name,country,teaching,international,research,citations,income,num_students,student_staff_ratio,international_students,female_male_ratio,year,females,males,ratio_male_to_female
1,2.0,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,2243.0,6.9,0.27,33 : 67,2011,33.0,67.0,2.030303
2,3.0,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,11074.0,9.0,0.33,37 : 63,2011,37.0,63.0,1.702703
3,4.0,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,15596.0,7.8,0.22,42 : 58,2011,42.0,58.0,1.380952
5,6.0,University of Cambridge,United Kingdom,90.5,77.7,94.1,94.0,57.0,18812.0,11.8,0.34,46 : 54,2011,46.0,54.0,1.173913
6,6.0,University of Oxford,United Kingdom,88.2,77.2,93.9,95.1,73.5,19919.0,11.6,0.34,46 : 54,2011,46.0,54.0,1.173913


------------------------------------------- ACCURACY ------------------------------------------------------------
------------------------------------------- ACCURACY TOP10 ------------------------------------------------------
Logistic regression TOP10, k-fold 10 - Accuracy 94.292% (9.137%)
------------------------------------------- ACCURACY TOP50 ------------------------------------------------------
Logistic regression TOP10, k-fold 10 - Accuracy 91.767% (9.613%)

------------------------------------------- LOGLOSS ------------------------------------------------------------
------------------------------------------- LOGLOSS TOP10 ------------------------------------------------------
Logistic regression, k-fold 4 - Logloss -0.269 (0.249)
------------------------------------------- LOGLOSS TOP50 ------------------------------------------------------
Logistic regression, k-fold 4 - Logloss -0.285 (0.198)

------------------------------------------- AUC -----------------------------

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Confusion Matrix
[[613   1]
 [ 36   3]]

Accuracy 94.33384
Accuracy check with conf. matrix 94.33384
------------------------------------------- CONFUSION MATRIX TOP50 --------------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Confusion Matrix
[[550   8]
 [ 37  58]]

Accuracy 93.10873
Accuracy check with conf. matrix 93.10873

------------------------------------------- CLASSIFICATION REPORT ----------------------------------------------------
------------------------------------------- CLASSIFICATION REPORT TOP10 ----------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 94.33384

              precision    recall  f1-score   support

           0    0.94453   0.99837   0.97070       614
           1    0.75000   0.07692   0.13953        39

    accuracy                        0.94334       653
   macro avg    0.84727   0.53765   0.55512       653
weighted avg    0.93291   0.94334   0.92106       653

------------------------------------------- CLASSIFICATION REPORT TOP50 ----------------------------------------------


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 93.10873

              precision    recall  f1-score   support

           0    0.93697   0.98566   0.96070       558
           1    0.87879   0.61053   0.72050        95

    accuracy                        0.93109       653
   macro avg    0.90788   0.79809   0.84060       653
weighted avg    0.92850   0.93109   0.92575       653


Discussion of differences in the cell below


------------------------------------------------------------------------------------------------------------------------------
-------------------------------------DISCUSS DIFFERENCES---------------------------------------------------------------------

As expected, the ACCURACY of the logistic regression when predicting Top10 (94%) is higher than for Top50 (91%), and the standard deviation for this dataset is greater than for the Shanghai one, although the difference betweetn standard deviations  is lower in this case (in relative terms).
The LOGLOSS obtained are quite similar, although is seems that the failed predictions in the Top10 are better than the ones of the Top50 (which are less negative, therefore the logarithm of the sum is closer to one instead of zero.
The AUC in this case is greater for the TOP50, and for this case it also presents a lower standard deviation. Still, both AUC far above the random curve and close to one.
The CONFUSION MATRIX shows that the accuracy in the case of the Top10 (94%) has been slightly better than for the Top50 (93%), and the CLASSIFICATION REPORT provides additional metrics.


In [10]:
len(Y)

2174

<h1> REGRESSION METRICS </h1>

We will review the three most common regression metrics,
<blockquote>
1) Mean Absolute Error (MAE).
<br>
2) Mean Square Error (MSE).
<br>
3) $R^2$


<img src="boston.jpg" width=1024>

<img src="Boston-Dataset-char.png">

In [11]:
# Load the Boston Housing dataset and separate input and output components 

from numpy import set_printoptions
set_printoptions(precision=3)

filename="HousingData.csv"
b_housing=pd.read_csv(filename)
b_housing.head()

b_housing.fillna(0,inplace=True) # we have NaN

# First we separate into input and output components
array=b_housing.values
X=array[:,0:13]
Y=array[:,13]
np.set_printoptions(suppress=True)
X
pd.DataFrame(X).head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


array([[  0.006,  18.   ,   2.31 , ...,  15.3  , 396.9  ,   4.98 ],
       [  0.027,   0.   ,   7.07 , ...,  17.8  , 396.9  ,   9.14 ],
       [  0.027,   0.   ,   7.07 , ...,  17.8  , 392.83 ,   4.03 ],
       ...,
       [  0.061,   0.   ,  11.93 , ...,  21.   , 396.9  ,   5.64 ],
       [  0.11 ,   0.   ,  11.93 , ...,  21.   , 393.45 ,   6.48 ],
       [  0.047,   0.   ,  11.93 , ...,  21.   , 396.9  ,   7.88 ]])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,0.00


<h1>Mean Absolute Error</h1>

The MAE (Mean Absolute Error) is the sum of the absolute differences between the actual values and the predictions.

It provides an idea of the magnitude of the error but not of its direction. A 0 indicates a perfect prediction and like logloss this metric is inverted by the cross_val_score() function.


In [12]:
# Mean Absolute Error

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression

# KFold
kfold = KFold(n_splits=10, random_state=7)

#model
model = LinearRegression()

scoring = "neg_mean_absolute_error"
res = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)

print(f'Boston Housing - Linear Regression, MAE: {res.mean():.3f} ({res.std():.3f})')

Boston Housing - Linear Regression, MAE: -3.975 (2.152)


<h1>Mean Squared Error</h1>

The idea of the MSE is the same of the MAE but we square the value in order to obtain always a positive value. Again, it provides an idea of the magnitude but not of the direction. 

Many times we use the RMSE (Root Mean Squared Error) in order to convert the units back to the original units of the output variable.   

Again this metric is inverted so results are increasing (scores that should be minimized are presented as negative while the ones that should be maximized as positive). 


In [13]:
# Mean Squared Error

import math 

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression

# KFold
kfold = KFold(n_splits=10, random_state=7)

#model
model = LinearRegression()

scoring = "neg_mean_squared_error"
res = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)

print(f'Boston Housing - Linear Regression, MSE: {res.mean():.3f} ({res.std():.3f})')
print(f'Boston Housing - Linear Regression, MSE: {math.sqrt(abs(res.mean())):.3f} ({math.sqrt(res.std()):.3f})')

Boston Housing - Linear Regression, MSE: -34.090 (44.046)
Boston Housing - Linear Regression, MSE: 5.839 (6.637)


<h1>$R^2$</h1>

The coefficient of determination $R^2$ provides an indication of the goodness of the predictions.

It's a value of 0 and 1 for non-fit and perfect fit respectively. A value closer to 0 and less than 0.5 indicates a poor fit.


In [14]:
# R2

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression

# KFold
kfold = KFold(n_splits=10, random_state=7)

#model
model = LinearRegression()

scoring = "r2"
res = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)

print(f'Boston Housing - Linear Regression, R2: {res.mean():.3f} ({res.std():.3f})')


Boston Housing - Linear Regression, R2: 0.243 (0.573)
